### Response to Reviews
# Phylogenetic comparative methods

### using [BayesTraits](http://www.evolution.rdg.ac.uk/BayesTraits.html) 3.0.1

#### Motif set 1: (Sp-D, Sp-L, motif Z)
#### Motif set 2: (motif X, motif Y, Maze)
#### All fish (2,837 genera for which trait data are available)


In [ ]:
version

In [ ]:
library(tidyverse)
library(ggpubr)
library(RColorBrewer)

packageVersion("tidyverse")
packageVersion("ggpubr")
packageVersion("RColorBrewer")

source("PCM_misc.R")

In [ ]:
get_MgLh <- function(target_order, ptns, model, hp_exp1, hp_exp2, rep) {
    fbase <- str_c(target_order, "state", ptns, sep="_")
    modelpar <- str_c(model, "rjMCMC_SCT_HPexp", hp_exp1, hp_exp2, "rep", rep, sep="_")
    stones_file <- str_c("./logs/", fbase, "_", modelpar, ".Stones.txt", sep = "")
    
    if (! file.exists(stones_file)) {
        print(str_c("file not found: ", stones_file))
        return(NA_real_)
    }
    
    MgLh <- NA_real_
    tryCatch({
        df_MgLh <- read_tsv(stones_file, skip = 107, col_names = FALSE, col_types="cd")
        MgLh <- df_MgLh[[1, 2]]
        }, error=function(e){}
    )
    return(MgLh)
}

get_df_MCMC <- function(target_order, ptns, model, hp_exp1, hp_exp2, rep) {
    fbase <- str_c(target_order, "state", ptns, sep="_")
    modelpar <- str_c(model, "rjMCMC_SCT_HPexp", hp_exp1, hp_exp2, "rep", rep, sep="_")
    log_file <- str_c("./logs/", fbase, "_", modelpar, ".Log.txt", sep = "")
    
    if (! file.exists(log_file)) {
        print(str_c("file not found: ", log_file))
        return(NA_real_)
    }

    colsp <- cols(.default = col_double(),
                  `Model string` = col_character())

    df_MCMC <- NA
    tryCatch({
        start_line <- grep("^Iteration\tLh", read_lines(log_file))
        # print(str_c(start_line, " lines skipped"))
        df_MCMC <- read_tsv(log_file,
                            skip = (start_line - 1),
                            col_types = colsp)
        df_MCMC <- df_MCMC %>%
            select(-starts_with("X"))
        
        }, error=function(e){ }
    )
    
    return(df_MCMC)
}

get_mean_rates <- function(target_order, ptns, model, rep, row, column) {
    df <- get_df_MCMC(target_order, ptns, model, 0, 10, rep)
    return(rateMatMean(df)[row, column])
}

In [ ]:
ptns_list <- c("Sp_D_Sp_L_Maze",
               "Sp_D_Sp_L_Eyes",
               "Sp_D_Sp_L_Sddl",
               "Sp_D_Sp_L_St_H",
               "Sp_D_Sp_L_St_V",
               "Sp_D_Sp_L_St_D",
               "Sp_D_Sp_L_Bltc",
               "Sp_D_Sp_L_Mono",
               "Sp_D_Sp_L_Area",
               "St_H_St_V_Maze",
               "St_H_Sp_D_Maze",
               "St_H_Sp_L_Maze",
               "St_V_Sp_D_Maze",
               "St_V_Sp_L_Maze")
hp_exp1 <- 0
hp_exp2 <- 10

models = c("IND", "indZ", "indX", "indY", "DEP")

get_df_model_rep <- function(model, ptns, rep) {
    df <- tibble("order"="AllFish_mini")
    df <- df %>%
    mutate(
        MgLh = map_dbl(order, get_MgLh, ptns = ptns, model = model,
                       hp_exp1 = hp_exp1, hp_exp2 = hp_exp2,
                       rep = rep),
        ptns = ptns,
        model = model,
        rep = rep
    )
    return(df)
}

get_df_all_rep <- function(ptns) {
    df_rep1 <- bind_rows(map(models, get_df_model_rep, ptns, 1))
    df_rep2 <- bind_rows(map(models, get_df_model_rep, ptns, 2))
    df_rep3 <- bind_rows(map(models, get_df_model_rep, ptns, 3))
    df_all_rep <- bind_rows(df_rep1, df_rep2, df_rep3)
    return(df_all_rep)
}

df_Sp_D_Sp_L_Maze <- get_df_all_rep(ptns_list[1])
df_Sp_D_Sp_L_Eyes <- get_df_all_rep(ptns_list[2])
df_Sp_D_Sp_L_Sddl <- get_df_all_rep(ptns_list[3])
df_Sp_D_Sp_L_St_H <- get_df_all_rep(ptns_list[4])
df_Sp_D_Sp_L_St_V <- get_df_all_rep(ptns_list[5])
df_Sp_D_Sp_L_St_D <- get_df_all_rep(ptns_list[6])
df_Sp_D_Sp_L_Bltc <- get_df_all_rep(ptns_list[7])
df_Sp_D_Sp_L_Mono <- get_df_all_rep(ptns_list[8])
df_Sp_D_Sp_L_Area <- get_df_all_rep(ptns_list[9])

df_St_H_St_V_Maze <- get_df_all_rep(ptns_list[10])
df_St_H_Sp_D_Maze <- get_df_all_rep(ptns_list[11])
df_St_H_Sp_L_Maze <- get_df_all_rep(ptns_list[12])
df_St_V_Sp_D_Maze <- get_df_all_rep(ptns_list[13])
df_St_V_Sp_L_Maze <- get_df_all_rep(ptns_list[14])

df_allfish <- bind_rows(df_Sp_D_Sp_L_Maze,
                        df_Sp_D_Sp_L_Eyes,
                        df_Sp_D_Sp_L_Sddl,
                        df_Sp_D_Sp_L_St_H,
                        df_Sp_D_Sp_L_St_V,
                        df_Sp_D_Sp_L_St_D,
                        df_Sp_D_Sp_L_Bltc,
                        df_Sp_D_Sp_L_Mono,
                        df_Sp_D_Sp_L_Area,
                        df_St_H_St_V_Maze,
                        df_St_H_Sp_D_Maze,
                        df_St_H_Sp_L_Maze,
                        df_St_V_Sp_D_Maze,
                        df_St_V_Sp_L_Maze)

df_allfish

In [ ]:
df_BF_odrs <- df_allfish %>%
    group_by(order, ptns, model) %>%
    summarize(MgLh = mean(MgLh)) %>%
    pivot_wider(names_from = model, values_from = MgLh) %>%
    mutate(
        DEP_IND = 2 * (DEP - IND),
        indZ_IND = 2 * (indZ - IND),
        DEP_indZ = 2 * (DEP - indZ),
        indX_IND = 2 * (indX - IND),
        DEP_indX = 2 * (DEP - indX),
        indY_IND = 2 * (indY - IND),
        DEP_indY = 2 * (DEP - indY)
    ) %>%
    pivot_longer(c(DEP_IND,
                   indZ_IND,
                   DEP_indZ,
                   indX_IND,
                   DEP_indX,
                   indY_IND,
                   DEP_indY), names_to = "comparison", values_to = "logBF" ) %>%
    mutate(ind = case_when(
                str_detect(comparison, "indZ") ~ "indZ",
                str_detect(comparison, "indX") ~ "indX",
                str_detect(comparison, "indY") ~ "indY",
                TRUE ~ "XYZ")) %>%
    ungroup()

df_BF_odrs


In [ ]:
options(repr.plot.width = 7, repr.plot.height = 4)

exp_DEP_indZ <- expression(paste(" DEP-ind", italic("Z")))
exp_indZ_IND <- expression(paste(" ind", italic("Z"), "-IND"))
cols <- brewer.pal(3, "Pastel1")

df_BF_odrs %>%
    filter(str_detect(ptns, "Sp_D_Sp_L_")) %>%
    filter(ind %in% c("indZ")) %>%

    mutate(dep_ind = case_when(
        str_detect(comparison, "DEP_indZ") ~ "  DEP - indZ",
        str_detect(comparison, "indZ_IND") ~ "  indZ - IND"
    )) %>%

    mutate(ptns2 = case_when(
            ptns == "Sp_D_Sp_L_Mono" ~ "(Sp-D, Sp-L, Mono)",
            ptns == "Sp_D_Sp_L_Bltc" ~ "(Sp-D, Sp-L, Bltc)",
            ptns == "Sp_D_Sp_L_Maze" ~ "(Sp-D, Sp-L, Maze)",
            ptns == "Sp_D_Sp_L_St_H" ~ "(Sp-D, Sp-L, St-H)",
            ptns == "Sp_D_Sp_L_St_D" ~ "(Sp-D, Sp-L, St-D)",
            ptns == "Sp_D_Sp_L_St_V" ~ "(Sp-D, Sp-L, St-V)",
            ptns == "Sp_D_Sp_L_Sddl" ~ "(Sp-D, Sp-L, Sddl)",
            ptns == "Sp_D_Sp_L_Eyes" ~ "(Sp-D, Sp-L, Eyes)",
            ptns == "Sp_D_Sp_L_Area" ~ "(Sp-D, Sp-L, Area)",
            TRUE ~ ptns
    )) %>%

    mutate(ptns2 = fct_relevel(ptns2,
                               "(Sp-D, Sp-L, Mono)",
                               "(Sp-D, Sp-L, Bltc)",
                               "(Sp-D, Sp-L, Maze)",
                               "(Sp-D, Sp-L, St-H)",
                               "(Sp-D, Sp-L, St-D)",
                               "(Sp-D, Sp-L, St-V)",
                               "(Sp-D, Sp-L, Sddl)",
                               "(Sp-D, Sp-L, Eyes)",
                               "(Sp-D, Sp-L, Area)"
    )) %>%

    ggplot(aes(x = fct_reorder(ptns2, desc(ptns2)), y = logBF, fill = dep_ind)) +
        geom_bar(width = 0.7, position = "stack", stat = "identity") +
        coord_flip() +
        labs(x = "", y = "Log Bayes factor", fill = "Model comparison") + 
        scale_fill_manual(labels = c(exp_DEP_indZ, exp_indZ_IND),
                          values = cols) +
                          # values = c("lightpink", "lightsteelblue")) +
        # scale_fill_brewer(palette = "Pastel1") +
        # scale_fill_discrete(name = "Model comparison") +
        # scale_fill_viridis_d(name = "Model comparison", direction = -1) +
        # theme_light() +
        theme_classic() +
        # theme_minimal() + # theme(panel.grid=element_blank()) +
        theme(axis.text.y = element_text(family = "mono", size = 13, color = "black"),
              axis.title.x = element_text(size = 14, color = "black"),
              axis.text.x = element_text(size = 9, color = "black"),
              legend.title = element_text(size = 12, color = "black"),
              legend.text = element_text(size = 12, family = "mono", color = "black")) -> plots

plots

plots %>%
    ggsave(file = "LogBF_Sp_DL-motifZ.pdf", device = cairo_pdf, width = 7, height = 4)

        

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 4)

exp_DEP_indZ <- expression(paste("DEP-ind[", italic("XYZ"), "]"))
exp_indZ_IND <- expression(paste("ind[", italic("XYZ"), "]-IND"))
exp_indX <- expression(paste(italic("X")))
exp_indY <- expression(paste(italic("Y")))
exp_indZ <- expression(paste(italic("Z")))
cols <- brewer.pal(3, "Pastel1")

df_BF_odrs %>%
    filter(str_detect(ptns, "_Maze")) %>%
    filter(ind %in% c("indZ", "indX", "indY")) %>%

    mutate(dep_ind = case_when(
        str_detect(comparison, "DEP_IND") ~ "DEP - IND",
        str_detect(comparison, "DEP_ind[XYZ]") ~ "  DEP - ind[XYZ]",
        str_detect(comparison, "ind[XYZ]_IND") ~ "  ind[XYZ] - IND"
    )) %>%

    mutate(ptns2 = case_when(
            ptns == "Sp_D_Sp_L_Maze" ~ "(Sp-D, Sp-L, Maze)",
            ptns == "St_H_St_V_Maze" ~ "(St-H, St-V, Maze)",
            ptns == "St_H_Sp_D_Maze" ~ "(St-H, Sp-D, Maze)",
            ptns == "St_H_Sp_L_Maze" ~ "(St-H, Sp-L, Maze)",
            ptns == "St_V_Sp_D_Maze" ~ "(St-V, Sp-D, Maze)",
            ptns == "St_V_Sp_L_Maze" ~ "(St-V, Sp-L, Maze)",
            TRUE ~ ptns
    )) %>%

    mutate(ptns2 = fct_relevel(ptns2,
                               "(Sp-D, Sp-L, Maze)",
                               "(St-H, St-V, Maze)",
                               "(St-H, Sp-D, Maze)",
                               "(St-H, Sp-L, Maze)",
                               "(St-V, Sp-D, Maze)",
                               "(St-V, Sp-L, Maze)"
    )) %>%

    ggplot(aes(x = fct_reorder(ind, desc(ind)), y = logBF, fill = dep_ind)) +
        geom_bar(position = "stack", stat = "identity") +        
        facet_grid(rows = vars(ptns2),
                   switch = "y") +
        coord_flip() +
        labs(x = "", y = "Log Bayes factor", fill = "Model comparison") +
        scale_x_discrete(labels = c(exp_indZ, exp_indY, exp_indX)) +
        scale_fill_manual(labels = c(exp_DEP_indZ, exp_indZ_IND),
                          values = cols) +
                          # values = c("lightpink", "lightsteelblue")) +
        # scale_fill_brewer(palette = "Pastel1") +
        # scale_fill_discrete(name = "Model comparison") +
        # theme_light() +
        # theme_bw() +
        theme_classic() +
        # theme_void() +
        # theme_minimal() + # theme(panel.grid=element_blank()) +
        theme(strip.text.y = element_text(angle = 180, family = "mono", size = 13, color = "black"),
              axis.text.y = element_text(size = 12, family = "mono", color = "black"),
              axis.title.x = element_text(size = 14, color = "black"),
              axis.text.x = element_text(size = 9, color = "black"),
              legend.title = element_text(size = 12, color = "black"),
              legend.text = element_text(size = 12, family = "mono", color = "black"),
              strip.background = element_blank(),
              strip.placement = "outside")  -> plots

plots

plots %>%
    ggsave(file = "LogBF_motifXY-Maze.pdf", device = cairo_pdf, width = 7.5, height = 4)
        

In [ ]:
df_rates_odrs <- df_allfish %>%
    filter(model == "DEP") %>%
    mutate(rates = pmap(list(order, ptns, model, rep), get_mean_rates))


In [ ]:
df_rates <- df_rates_odrs %>%
    mutate(
        q14 = map_dbl(rates, ~ .x[1, 4]),
        q26 = map_dbl(rates, ~ .x[2, 6]),
        q37 = map_dbl(rates, ~ .x[3, 7]),
        q58 = map_dbl(rates, ~ .x[5, 8]),
        q41 = map_dbl(rates, ~ .x[4, 1]),
        q62 = map_dbl(rates, ~ .x[6, 2]),
        q73 = map_dbl(rates, ~ .x[7, 3]),
        q85 = map_dbl(rates, ~ .x[8, 5])
    ) %>%
    select(order, MgLh, ptns, model, rep, q14:q85) %>%
    pivot_longer(cols = c(q14, q26, q37, q58, q41, q62, q73, q85), names_to = "qij", values_to = "rate")

df_rates

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 5)

exp_tr_Z_gain <- expression(paste("Transition rate (motif ", italic("Z"), " gain)"))
exp_tr_Z_loss <- expression(paste("Transition rate (motif ", italic("Z"), " loss)"))
exp_ptnsXYZ <- expression(paste("Motif set (", italic("X"), ", ", italic("Y"), ", ", italic("Z"), ")"))

palette <- c("silver",    # Mono
             "dimgray",   # Bltc
             # 'mediumseagreen',     # Sp_D
             ## '#35A16B',   # Sp_D
             # 'tomato',     # Sp_L
             ## 'orange',     # Sp_L
             ## '#FAF500',  # Maze
             "gold",        # Maze
             "turquoise",   # St_H
             "royalblue",   # St_D
             "darkslateblue",  # St_V
             "mediumorchid",   # Sddl
             "pink",        # Eyes
             "sienna")      # Area

palette_hex <- c('#c0c0c0',  # Mono
                 '#696969',  # Bltc
                 # '#3cb371',  # Sp_D
                 # '#ff6347',  # Sp_L
                 '#ffd700',  # Maze
                 '#40e0d0',  # St_H
                 '#4169e1',  # St_D
                 '#483d8b',  # St_V
                 '#ba55d3',  # Sddl
                 '#ffc0cb',  # Eyes
                 '#a0522d')  # Area

df_Z <- df_rates %>%
    filter(model == "DEP") %>%
    filter(str_detect(ptns, "Sp_D_Sp_L_")) %>%

    mutate(ptns2 = case_when(
            ptns == "Sp_D_Sp_L_Mono" ~ "(Sp-D, Sp-L, Mono)",
            ptns == "Sp_D_Sp_L_Bltc" ~ "(Sp-D, Sp-L, Bltc)",
            ptns == "Sp_D_Sp_L_Maze" ~ "(Sp-D, Sp-L, Maze)",
            ptns == "Sp_D_Sp_L_St_H" ~ "(Sp-D, Sp-L, St-H)",
            ptns == "Sp_D_Sp_L_St_D" ~ "(Sp-D, Sp-L, St-D)",
            ptns == "Sp_D_Sp_L_St_V" ~ "(Sp-D, Sp-L, St-V)",
            ptns == "Sp_D_Sp_L_Sddl" ~ "(Sp-D, Sp-L, Sddl)",
            ptns == "Sp_D_Sp_L_Eyes" ~ "(Sp-D, Sp-L, Eyes)",
            ptns == "Sp_D_Sp_L_Area" ~ "(Sp-D, Sp-L, Area)",
            TRUE ~ ptns
    )) %>%

    mutate(ptns2 = fct_relevel(ptns2,
                               "(Sp-D, Sp-L, Mono)",
                               "(Sp-D, Sp-L, Bltc)",
                               "(Sp-D, Sp-L, Maze)",
                               "(Sp-D, Sp-L, St-H)",
                               "(Sp-D, Sp-L, St-D)",
                               "(Sp-D, Sp-L, St-V)",
                               "(Sp-D, Sp-L, Sddl)",
                               "(Sp-D, Sp-L, Eyes)",
                               "(Sp-D, Sp-L, Area)"
    )) %>%

    group_by(order, ptns2, qij) %>%
    summarize(mean_rate = mean(rate)) %>%
    ungroup() %>%
    pivot_wider(names_from = qij, values_from = mean_rate) %>%
    drop_na()

Zgain <- df_Z %>%
    pivot_longer(cols = c(q14, q26, q37, q58), names_to = "qij", values_to = "mean_rate") %>%

    ggplot(aes(x = qij, y = mean_rate)) +
        geom_line(aes(color = ptns2, group = ptns2), size = 1, alpha = 0.8) +
        geom_point(aes(color = ptns2, fill = ptns2, shape = ptns2, stroke = ptns2), size = 4) +
        scale_x_discrete(labels = c("a", "b", "c", "d")) +
        scale_y_continuous() +
        scale_color_manual(values = alpha(palette_hex, 0.8)) +
        scale_fill_manual(values = alpha(palette_hex, 0.8)) +
        scale_shape_manual(values = c(22, 20, 4, 22, 22, 22, 24, 21, 22)) +
        scale_discrete_manual(aesthetics = "stroke", values = c(1, 1, 2, 1, 1, 1, 1, 1, 1)) +
        theme_classic() +
        theme(axis.text.x = element_text(face = "italic", size = 22, color = "black"),
              axis.text.y = element_text(size = 10, color = "black"),
              axis.title.y = element_text(size = 18, color = "black"),
              legend.title = element_text(size = 13, color = "black"),
              legend.text = element_text(family = "mono", size = 12, color = "black"),
              legend.key.size = unit(1.2, "line"),
              plot.margin = unit(c(1, 1, 0, 0.7), "lines")) +
        guides(shape = guide_legend(override.aes = list(alpha = 1, linetype = 0))) +
        labs(x = "", y = exp_tr_Z_gain,
             color = exp_ptnsXYZ, 
             shape = exp_ptnsXYZ,
             fill = exp_ptnsXYZ,
             stroke = exp_ptnsXYZ)

Zloss <- df_Z %>%
    pivot_longer(cols = c(q41, q62, q73, q85), names_to = "qij", values_to = "mean_rate") %>%

    ggplot(aes(x = qij, y = mean_rate)) +
        geom_line(aes(color = ptns2, group = ptns2), size = 1, alpha = 0.8) +
        geom_point(aes(color = ptns2, fill = ptns2, shape = ptns2, stroke = ptns2), size = 4) +
        scale_x_discrete(labels = c("e", "f", "g", "h")) +
        scale_y_continuous(breaks = seq(0.2, 0.8, 0.2)) +
        scale_color_manual(values = alpha(palette_hex, 0.8)) +
        scale_fill_manual(values = alpha(palette_hex, 0.8)) +
        scale_shape_manual(values = c(22, 20, 4, 22, 22, 22, 24, 21, 22)) +
        scale_discrete_manual(aesthetics = "stroke", values = c(1, 1, 2, 1, 1, 1, 1, 1, 1)) +
        theme_classic() +
        theme(axis.text.x = element_text(face = "italic", size = 22, color = "black"),
              axis.text.y = element_text(size = 10, color = "black"),
              axis.title.y = element_text(size = 18, color = "black"),
              legend.title = element_text(size = 13, color = "black"),
              legend.text = element_text(family = "mono", size = 12, color = "black"),
              legend.key.size = unit(1.2, "line"),
              plot.margin = unit(c(1, 1, 0, 0.7), "lines")) +
        guides(shape = guide_legend(override.aes = list(alpha = 1, linetype = 0))) +
        labs(x = "", y = exp_tr_Z_loss,
             color = exp_ptnsXYZ, 
             shape = exp_ptnsXYZ,
             fill = exp_ptnsXYZ,
             stroke = exp_ptnsXYZ)

plots <- ggarrange(Zgain, Zloss,
                   ncol = 2,
                   common.legend = TRUE, legend = "right")

plots

plots %>%
    ggsave(file = "pcm_Sp_DL-motifZ_gain_loss_rates.pdf", device = cairo_pdf, width = 9, height = 5)
        

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 5)

exp_tr_Z_gain <- expression(paste("Transition rate (Maze gain)"))
exp_tr_Z_loss <- expression(paste("Transition rate (Maze loss)"))
exp_ptnsXYZ <- expression(paste("Motif set (", italic("X"), ", ", italic("Y"), ", ", italic("Z"), ")"))

palette_hex <- c('#ffd700',  # (Sp_D, Sp_L, Maze)
                 '#',  # (Sp_D, Sp_L, Maze)
                 '#',  # (Sp_D, Sp_L, Maze)
                 '#',  # (Sp_D, Sp_L, Maze)
                 '#',  # (Sp_D, Sp_L, Maze)
                 '#')  # (Sp_D, Sp_L, Maze)

df_Z <- df_rates %>%
    filter(model == "DEP") %>%
    filter(str_detect(ptns, "_Maze")) %>%

    mutate(ptns2 = case_when(
            ptns == "Sp_D_Sp_L_Maze" ~ "(Sp-D, Sp-L, Maze)",
            ptns == "St_H_St_V_Maze" ~ "(St-H, St-V, Maze)",
            ptns == "St_H_Sp_D_Maze" ~ "(St-H, Sp-D, Maze)",
            ptns == "St_H_Sp_L_Maze" ~ "(St-H, Sp-L, Maze)",
            ptns == "St_V_Sp_D_Maze" ~ "(St-V, Sp-D, Maze)",
            ptns == "St_V_Sp_L_Maze" ~ "(St-V, Sp-L, Maze)",
            TRUE ~ ptns
    )) %>%

    mutate(ptns2 = fct_relevel(ptns2,
                               "(Sp-D, Sp-L, Maze)",
                               "(St-H, St-V, Maze)",
                               "(St-H, Sp-D, Maze)",
                               "(St-H, Sp-L, Maze)",
                               "(St-V, Sp-D, Maze)",
                               "(St-V, Sp-L, Maze)"
    )) %>%

    group_by(order, ptns2, qij) %>%
    summarize(mean_rate = mean(rate)) %>%
    ungroup() %>%
    pivot_wider(names_from = qij, values_from = mean_rate) %>%
    drop_na()

Zgain <- df_Z %>%
    pivot_longer(cols = c(q14, q26, q37, q58), names_to = "qij", values_to = "mean_rate") %>%

    ggplot(aes(x = qij, y = mean_rate)) +
        geom_line(aes(color = ptns2, group = ptns2), size = 1, alpha = 0.8) +
        geom_point(aes(color = ptns2, fill = ptns2, shape = ptns2, stroke = ptns2), size = 4) +
        scale_x_discrete(labels = c("a", "b", "c", "d")) +
        scale_y_continuous() +
        scale_shape_manual(values = c(4, 24, 22, 22, 21, 21)) +
        scale_discrete_manual(aesthetics = "stroke", values = c(2, 1, 1, 1, 1, 1)) +
        scale_color_viridis_d(direction = -1) +
        scale_fill_viridis_d(direction = -1) +
        theme_classic() +
        theme(axis.text.x = element_text(face = "italic", size = 22, color = "black"),
              axis.text.y = element_text(size = 10, color = "black"),
              axis.title.y = element_text(size = 18, color = "black"),
              legend.title = element_text(size = 13, color = "black"),
              legend.text = element_text(family = "mono", size = 12, color = "black"),
              legend.key.size = unit(1.2, "line"),
              plot.margin = unit(c(1, 1, 0, 0.7), "lines")) +
        guides(shape = guide_legend(override.aes = list(alpha = 1, linetype = 0))) +
        labs(x = "", y = exp_tr_Z_gain,
             color = exp_ptnsXYZ, 
             shape = exp_ptnsXYZ,
             fill = exp_ptnsXYZ,
             stroke = exp_ptnsXYZ)

Zloss <- df_Z %>%
    pivot_longer(cols = c(q41, q62, q73, q85), names_to = "qij", values_to = "mean_rate") %>%

    ggplot(aes(x = qij, y = mean_rate)) +
        geom_line(aes(color = ptns2, group = ptns2), size = 1, alpha = 0.8) +
        geom_point(aes(color = ptns2, fill = ptns2, shape = ptns2, stroke = ptns2), size = 4) +
        scale_x_discrete(labels = c("e", "f", "g", "h")) +
        scale_y_continuous() +
        scale_shape_manual(values = c(4, 24, 22, 22, 21, 21)) +
        scale_discrete_manual(aesthetics = "stroke", values = c(2, 1, 1, 1, 1, 1)) +
        scale_color_viridis_d(direction = -1) +
        scale_fill_viridis_d(direction = -1) +
        theme_classic() +
        theme(axis.text.x = element_text(face = "italic", size = 22, color = "black"),
              axis.text.y = element_text(size = 10, color = "black"),
              axis.title.y = element_text(size = 18, color = "black"),
              legend.title = element_text(size = 13, color = "black"),
              legend.text = element_text(family = "mono", size = 12, color = "black"),
              legend.key.size = unit(1.2, "line"),
              plot.margin = unit(c(1, 1, 0, 0.7), "lines")) +
        guides(shape = guide_legend(override.aes = list(alpha = 1, linetype = 0))) +
        labs(x = "", y = exp_tr_Z_loss,
             color = exp_ptnsXYZ, 
             shape = exp_ptnsXYZ,
             fill = exp_ptnsXYZ,
             stroke = exp_ptnsXYZ)

plots <- ggarrange(Zgain, Zloss,
                   ncol = 2,
                   common.legend = TRUE, legend = "right")

plots

plots %>%
    ggsave(file = "pcm_ptnXY-Maze_gain_loss_rates.pdf", device = cairo_pdf, width = 9, height = 5)
        